In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
df=pd.read_csv('../Data/HouseImformation.csv')

In [3]:
df['Area'] = df['Area'].str.replace(',', '').astype(float)

df=df.dropna()

isoOuter=IsolationForest(contamination=0.01)
df['OuterData']=isoOuter.fit_predict(df[["Area", "Price"]])

df=df.drop(index=df[df['OuterData'] == -1].index)

MeanPriceAddressOuter=df.groupby("Address")["Price"].mean()
df["MeanPriceA"] = df["Address"].map(MeanPriceAddressOuter)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
x_Data = scaler.fit_transform(df[['Area','Room','Parking','Warehouse','Elevator','MeanPriceA']])
y_Data=df[['Price']].values


In [5]:
mask= np.random.rand(len(df)) < 0.8

train=df[mask]
test=df[~mask]

x_train=np.asanyarray(train[['Area','Room','Parking','Warehouse','Elevator','MeanPriceA']])
y_train=np.asanyarray(train[['Price']])

x_test=np.asanyarray(test[['Area','Room','Parking','Warehouse','Elevator','MeanPriceA']])
y_test=np.asanyarray(test[['Price']])

In [6]:
from sklearn.model_selection import RepeatedKFold ,cross_val_score
from sklearn.linear_model import ElasticNet
model= ElasticNet(alpha=1.0,l1_ratio=0.8) # By testing different values, I found that 0.8 works best for l1_ratio But Standard deviation 

cv=RepeatedKFold(n_splits=10,n_repeats=3,random_state=1)

cScore=cross_val_score(model,x_Data,y_Data,cv=cv)


print("Mean Is: ",cScore.mean())
print("Standard deviation: ", np.std(cScore))

Mean Is:  0.7304615076764296
Standard deviation:  0.039239777429008166


In [7]:
model.fit(x_train,y_train)

y_pred=model.predict(x_test)

from sklearn.metrics import r2_score
print(f"Test R2 Score: {r2_score(y_test, y_pred):.2f}")


Test R2 Score: 0.78


In [8]:
joblib.dump(model,"../Models/ElasticNetModel")

['../Models/ElasticNetModel']

In [ ]:
model05= ElasticNet(alpha=1.0,l1_ratio=0.5)

model05.fit(x_train,y_train)

y_pred=model05.predict(x_test)

from sklearn.metrics import r2_score
print(f"Test R2 Score: {r2_score(y_test, y_pred):.2f}")

Test R2 Score: 0.74


c:\Users\Abolfazl\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.445e+22, tolerance: 1.105e+19
  model = cd_fast.enet_coordinate_descent(


<h2>Elastic Net Model With Polynomial Features Model</h2>

In [14]:
pip=Pipeline([
    ('scale',StandardScaler()),
    ('poly',PolynomialFeatures(degree=2)),
    ('elasticNet',ElasticNet(alpha=1.0,l1_ratio=0.8))
])

cvP=RepeatedKFold(n_splits=10,n_repeats=3,random_state=1)
crossVal=cross_val_score(pip,x_Data,y_Data,cv=cvP)


print("Mean Is: ",crossVal.mean())
print("Standard deviation: ", np.std(crossVal))

Mean Is:  0.8169524862765452
Standard deviation:  0.02470721760763163


In [15]:
pip.fit(x_train,y_train)

y_predict=pip.predict(x_test)


from sklearn.metrics import r2_score
print(f"Test R2 Score: {r2_score(y_test, y_predict):.2f}")

Test R2 Score: 0.83


In [16]:
joblib.dump(pip,"../Models/ElasticNetModelWithPolynomialFeaturesModel")

['../Models/ElasticNetModelWithPolynomialFeaturesModel']